# Машинное обучение, ШАД, Минск
## Лабораторная работа 3. Линейные модели классификации и регрессии, валидация моделей.


**Правила:**

* Выполненную работу нужно отправить в соответствующее задание в личном кабинете
* Дедлайн **14 октября 23:59**. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Для сдачи задания нужно загрузить **ноутбук в формате `ipynb`** в ЛМС.
* Выполнять задание необходимо полностью самостоятельно.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Присылайте понятный и читаемый код. Если код не будет понятен проверяющему, оценка может быть снижена.
* Код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*


**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * прикрепив ее в ЛМС в форматах `pdf`, `png` или `jpg` *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image`;
  * в виде $\LaTeX$ в markdown-ячейках или в отдельном `pdf`-файле.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.

**Максимальное количество баллов за задание: 5 баллов.**

----

### Задание.

Реализуйте логистическую регрессию с $L_2$ регуляризацией для поиска оценки параметров с помощью стохастического mini-batch градиентного спуска (SGD).

In [80]:
import numpy as np
from scipy.special import expit


class LogisticRegression():
    '''
    Модель логистической регрессии. Имеет следующие гиперпараметры:

    :param alpha: параметр регуляризации. 
                  Если равно 0, то регуляризация не происходит.
    :param lr: константа, на которую домножаем градиент при обучении
    :param max_iter: ограничение на кол-во итераций
    :param fit_intercept: указывает, следует ли добавить константу в признаки
    '''

    def __init__(self, alpha=0, lr=0.5, max_iter=1e5,
                 fit_intercept=True):
        '''Создает модель и инициализирует параметры.'''

        self.alpha = alpha
        self.lr = lr
        self.max_iter = max_iter
        self.fit_intercept = fit_intercept

    @staticmethod
    def _sigmoid(x):
        # используйте scipy.special.expit
        return expit(x)

    def _add_intercept(self, X):
        '''
        Добавляем свободный коэффициент к нашей модели. 
        Это происходит путем добавления вектора из 1 к исходной матрице.

        :param X: исходная матрица признаков
        :return: матрица X с добавленным свободным коэффициентов
        '''

        X_copy = np.full((X.shape[0], X.shape[1] + 1), fill_value=1.)
        X_copy[:, :-1] = X

        return X_copy

    def fit(self, X, Y):
        '''
        Обучает модель логистической регресии с помощью SGD,
        пока не выполнится self.max_iter итераций.

        :param X: матрица признаков
        :param Y: истинные метки
        '''

        assert X.shape[0] == Y.shape[0]

        if self.fit_intercept:  # добавляем свободный коэфициент
            X_copy = self._add_intercept(X)
        else:
            X_copy = X.copy()

        n_samples, n_features = X_copy.shape
        self.weights = np.zeros(n_features)

        self.batch_size = 32

        for _ in range(int(self.max_iter)):
            indices = np.random.permutation(n_samples)
            for i in range(0, n_samples, self.batch_size):
                batch_indices = indices[i:i + self.batch_size]
                X_batch = X_copy[batch_indices]
                Y_batch = Y[batch_indices]

                logit = np.dot(X_batch, self.weights)
                y_pred = self._sigmoid(logit)

                grad = -1 * np.dot(X_batch.T, Y_batch - y_pred)
                if self.fit_intercept:
                    regul = 2 * self.alpha * self.weights[:-1]
                else:
                    regul = 2 * self.alpha * self.weights

                self.weights -= self.lr * grad
                if self.fit_intercept:
                    self.weights[:-1] -= self.lr * regul
                else:
                    self.weights -= self.lr * regul

        self.coef_ = self.weights[:-1] if self.fit_intercept else self.weights # коэффициенты модели
        self.intercept_ = self.weights[-1] if self.fit_intercept else 0  # свободный коэффициент
        # self.weights состоит из коэффициентов модели и свободного члена
        return self

    def predict(self, X):
        '''
        Применяет обученную модель к данным 
        и возвращает точечное предсказание (оценку класса).

        :param X: матрица признаков
        :return: предсказание с размерностью (n_test, )
        '''

        if self.fit_intercept:
            X_copy = self._add_intercept(X)
        else:
            X_copy = X.copy()

        assert X_copy.shape[1] == self.weights.shape[0]

        predictions = self._sigmoid(np.dot(X_copy, self.weights)) >= 0.5

        return predictions.astype(int)

    def predict_proba(self, X):
        '''
        Применяет обученную модель к данным
        и возвращает предсказание вероятности классов 0 и 1.

        :param X: матрица признаков
        :return: вероятности предсказания с размерностью (n_test, 2)
        '''

        if self.fit_intercept:
            X_copy = self._add_intercept(X)
        else:
            X_copy = X.copy()

        assert X_copy.shape[1] == self.weights.shape[0]

        prob_predictions = self._sigmoid(np.dot(X_copy, self.weights))

        return np.column_stack([1 - prob_predictions, prob_predictions])

Рассмотрим игрушечный датасет на $30$ признаков `load_breast_cancer` из библиотеки `sklearn`. Это относительно простой для бинарной классификации датасет по диагностике рака молочной железы.

Ради интереса можно прочитать описание признаков.

In [81]:
from sklearn.datasets import load_breast_cancer


dataset = load_breast_cancer()
dataset['DESCR'].split('\n')[11:31]

[':Attribute Information:',
 '    - radius (mean of distances from center to points on the perimeter)',
 '    - texture (standard deviation of gray-scale values)',
 '    - perimeter',
 '    - area',
 '    - smoothness (local variation in radius lengths)',
 '    - compactness (perimeter^2 / area - 1.0)',
 '    - concavity (severity of concave portions of the contour)',
 '    - concave points (number of concave portions of the contour)',
 '    - symmetry',
 '    - fractal dimension ("coastline approximation" - 1)',
 '',
 '    The mean, standard error, and "worst" or largest (mean of the three',
 '    worst/largest values) of these features were computed for each image,',
 '    resulting in 30 features.  For instance, field 0 is Mean Radius, field',
 '    10 is Radius SE, field 20 is Worst Radius.',
 '',
 '    - class:',
 '            - WDBC-Malignant',
 '            - WDBC-Benign']

Разделим нашу выборку на обучающую и тестовую:

In [82]:
from sklearn.model_selection import train_test_split


X, Y = dataset['data'], dataset['target']

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((455, 30), (114, 30), (455,), (114,))

При использовании регуляризации данные необходимо нормализовать. Воспользуемся для этого классом `StandardScaler` из библиотеки `sklearn`. 

In [83]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

Теперь обучите модель логистической регрессии.

In [84]:
from sklearn.metrics import accuracy_score


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train_scaled, Y_train)

Y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(Y_test, Y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9386


На занятии обсуждали, что в нашей постановке задачи при сравнении выиграет модель с меньшим FN, ведь каждая не обнаруженная опухоль может стоить человеческой жизни. Чем меньше ложно отрицательных срабатываний, тем выше Recall модели, а значит разумно взять Recall в качестве целевой метрики. 

Построить модель с Recall = 1 довольно просто (Как?), но в ней не будет большого смысла, т.к., например, для нашей задачи отправление на доп. обследование может стоить дополнительных средств и времени специалистов, поэтому хотелось, чтобы наша модель имела неплохую точность. Какую метрику можно использовать, чтобы учесть и точность, и полноту?

Чтобы учитывать и точность (precision), и полноту (recall), можно использовать F1-score — это гармоническое среднее между ними.

$F_1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$

Выберите и посчитайте целевые метрики для нашей задачи.

In [85]:
from sklearn.metrics import recall_score, f1_score


recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

Recall: 0.9155
F1-score: 0.9489


Рассмотрите как влияет размер шага (`learning rate`) на качество модели. Обучите каждую модель одинаковое число итераций (например, 10000), а затем посчитайте качество. Сделайте выводы.

In [86]:
lrs = [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.3, 0.5, 0.7, 1, 2, 5, 10]

In [87]:
for lr in lrs:
    model = LogisticRegression(lr=lr)
    model.fit(X_train_scaled, Y_train)

    Y_pred = model.predict(X_test_scaled)

    recall = recall_score(Y_test, Y_pred)
    f1 = f1_score(Y_test, Y_pred)

    print(f'Learning rate: {lr}, Recall: {recall:.4f}, F1-score: {f1:.4f}')

Learning rate: 1e-05, Recall: 0.9859, F1-score: 0.9859
Learning rate: 0.0001, Recall: 0.9718, F1-score: 0.9787
Learning rate: 0.001, Recall: 0.9155, F1-score: 0.9489
Learning rate: 0.01, Recall: 0.9155, F1-score: 0.9489
Learning rate: 0.1, Recall: 0.9155, F1-score: 0.9489
Learning rate: 0.2, Recall: 0.9155, F1-score: 0.9489
Learning rate: 0.3, Recall: 0.9155, F1-score: 0.9489
Learning rate: 0.5, Recall: 0.9155, F1-score: 0.9489
Learning rate: 0.7, Recall: 0.9155, F1-score: 0.9489
Learning rate: 1, Recall: 0.9155, F1-score: 0.9489
Learning rate: 2, Recall: 0.9155, F1-score: 0.9489
Learning rate: 5, Recall: 0.9155, F1-score: 0.9489
Learning rate: 10, Recall: 0.9155, F1-score: 0.9489


Рассмотрите несколько моделей, в которых установите не менее 5-ти различных коэффициентов регуляризации, а также модель без регуляризатора. Сравните, влияет ли наличие регуляризации на качество, сделайте выводы. Под качеством подразумевается значение какой-либо выбранной вами метрики качества классификации.

In [88]:
alphas = [0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.3, 0.5, 0.7, 1, 2]

for alpha in alphas:
    model = LogisticRegression(alpha=alpha)
    model.fit(X_train_scaled, Y_train)

    Y_pred = model.predict(X_test_scaled)

    recall = recall_score(Y_test, Y_pred)
    f1 = f1_score(Y_test, Y_pred)

    print(f'Alpha: {alpha}, Recall: {recall:.4f}, F1-score: {f1:.4f}')

Alpha: 0, Recall: 0.9155, F1-score: 0.9489
Alpha: 1e-05, Recall: 0.9155, F1-score: 0.9489
Alpha: 0.0001, Recall: 0.9014, F1-score: 0.9412
Alpha: 0.001, Recall: 0.9577, F1-score: 0.9645
Alpha: 0.01, Recall: 0.9577, F1-score: 0.9645
Alpha: 0.1, Recall: 0.9718, F1-score: 0.9650
Alpha: 0.2, Recall: 0.9859, F1-score: 0.9722
Alpha: 0.3, Recall: 0.9718, F1-score: 0.9583
Alpha: 0.5, Recall: 0.9859, F1-score: 0.9655
Alpha: 0.7, Recall: 0.7606, F1-score: 0.8438
Alpha: 1, Recall: 1.0000, F1-score: 0.7676
Alpha: 2, Recall: 0.0423, F1-score: 0.0522


Выберите наилучшее значение коэффициента регуляризации с помощью кросс-валидации для двух подходов &mdash; `KFold` и `ShuffleSplit`. Используйте пять фолдов/разбиений.

In [89]:
from sklearn.model_selection import KFold, ShuffleSplit


def cross_val_logistic_regression(X, Y, alphas, cv):
    results = []

    for alpha in alphas:
        scores = []

        for train_idx, test_idx in cv.split(X):
            X_train, X_test = X[train_idx], X[test_idx]
            Y_train, Y_test = Y[train_idx], Y[test_idx]

            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            model = LogisticRegression(alpha=alpha)
            model.fit(X_train_scaled, Y_train)
            Y_pred = model.predict(X_test_scaled)

            score = f1_score(Y_test, Y_pred)

            scores.append(score)

        mean_score = np.mean(scores)
        results.append((alpha, mean_score))

    results.sort(key=lambda x: x[1], reverse=True)
    return results


kf = KFold(n_splits=5, shuffle=True, random_state=42)

kf_results = cross_val_logistic_regression(X, Y, alphas, cv=kf)
print("KFold results (sorted by F1-score):")
for alpha, score in kf_results:
    print(f'Alpha: {alpha}, F1-score: {score:.4f}')

ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

ss_results = cross_val_logistic_regression(X, Y, alphas, cv=ss)
print('ShuffleSplit results (sorted by F1-score):')
for alpha, score in ss_results:
    print(f'Alpha: {alpha}, F1_score: {score:.4f}')

KFold results (sorted by F1-score):
Alpha: 0.001, F1-score: 0.9689
Alpha: 0.01, F1-score: 0.9649
Alpha: 0, F1-score: 0.9642
Alpha: 1e-05, F1-score: 0.9642
Alpha: 0.0001, F1-score: 0.9629
Alpha: 0.1, F1-score: 0.9599
Alpha: 0.2, F1-score: 0.9466
Alpha: 0.3, F1-score: 0.9336
Alpha: 0.5, F1-score: 0.9185
Alpha: 0.7, F1-score: 0.8997
Alpha: 1, F1-score: 0.8959
Alpha: 2, F1-score: 0.1032
ShuffleSplit results (sorted by F1-score):
Alpha: 0.001, F1_score: 0.9800
Alpha: 0.01, F1_score: 0.9800
Alpha: 0.0001, F1_score: 0.9797
Alpha: 0, F1_score: 0.9768
Alpha: 1e-05, F1_score: 0.9768
Alpha: 0.1, F1_score: 0.9579
Alpha: 0.5, F1_score: 0.9425
Alpha: 0.2, F1_score: 0.9280
Alpha: 0.3, F1_score: 0.9213
Alpha: 0.7, F1_score: 0.8781
Alpha: 1, F1_score: 0.8086
Alpha: 2, F1_score: 0.0995


Для выбранного значения коэффициента регуляризации оцените дисперсию усредненного значения метрики качества на тестовых батчах. Для этого выполните кросс-валидацию достаточно много раз (не менее 100) и посчитайте выборочную дисперсию. Обратите внимание, что для стратегии `KFold` нужно на каждой итерации перемешивать данные, для этого можно указать `shuffle=True`.

Сравните эти две стратегии кросс-валидации. Какие их преимущества и недостатки?

In [ ]:
def multiple_cross_val_logistic_regression(X, Y, alpha, cv, n_iter=100):
    scores = []

    for _ in range(n_iter):
        iter_scores = []

        for train_idx, test_idx in cv.split(X):
            X_train, X_test = X[train_idx], X[test_idx]
            Y_train, Y_test = Y[train_idx], Y[test_idx]

            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            model = LogisticRegression(alpha=alpha)
            model.fit(X_train_scaled, Y_train)
            Y_pred = model.predict(X_test_scaled)

            score = f1_score(Y_test, Y_pred)
            iter_scores.append(score)

        mean_iter_score = np.mean(iter_scores)
        scores.append(mean_iter_score)

    return np.array(scores)

alpha = 0.001

kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf_scores = multiple_cross_val_logistic_regression(X, Y, alpha, cv=kf, n_iter=100)

ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
ss_scores = multiple_cross_val_logistic_regression(X, Y, alpha, cv=ss, n_iter=100)

kf_variance = np.var(kf_scores)
ss_variance = np.var(ss_scores)

print(f'Dispersion for KFold: {kf_variance:.6f}')
print(f'Dispersion for ShuffleSplit: {ss_variance:.6f}')

**Вывод:**

1. **KFold с перемешиванием:**
   - **Преимущества:** Каждая часть данных используется как для обучения, так и для тестирования, что делает оценку более точной. Перемешивание уменьшает возможное смещение данных.
   - **Недостатки:** Вычислительно более затратен, особенно при большом количестве данных. Может привести к большему разбросу оценок на меньших тестовых наборах.

2. **ShuffleSplit:**
   - **Преимущества:** Гибкость в настройке размера тестовой выборки. Более быстрый и менее затратный метод. Более стабильные оценки, так как тестовые выборки больше.
   - **Недостатки:** Некоторая выборка может не использоваться в обучении или тестировании. Возможны повторяющиеся выборки в разных разбиениях.

**Заключение:** KFold обеспечивает более точную оценку за счет использования всех данных, но медленнее. ShuffleSplit более гибкий и быстрый, но может вносить смещение в оценку.